In [1]:
import inspect
import justcause.learners
for name, obj in inspect.getmembers(justcause.learners):
    if inspect.isclass(obj):
            print(obj)

<class 'justcause.learners.tree.causal_forest.CausalForest'>
<class 'justcause.learners.ate.double_robust.DoubleRobustEstimator'>
<class 'justcause.learners.nn.dragonnet.DragonNet'>
<class 'justcause.learners.ate.propensity_weighting.PSWEstimator'>
<class 'justcause.learners.meta.rlearner.RLearner'>
<class 'justcause.learners.meta.slearner.SLearner'>
<class 'justcause.learners.meta.tlearner.TLearner'>
<class 'justcause.learners.meta.xlearner.XLearner'>


In [2]:
from justcause.data.generators import multi_expo_on_ihdp

# Load a list of 100 replications
#multi = multi_expo_on_ihdp(setting='multi-modal', n_replications=1000)

expo = multi_expo_on_ihdp(setting='exponential', n_replications=10000)

In [30]:
#from justcause.learners.utils import install_r_packages
 
from justcause.learners import SLearner, TLearner, DragonNet
 

from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
 
# Ensure 'grf' R package is installed in the environment
 
#install_r_packages(['grf'])
 
from sklearn.linear_model import LinearRegression 

class PolyLearner:
    def __init__(self, degree):
        self.degree = degree
        self.poly = PolynomialFeatures(degree)
        self.linReg = LinearRegression()
    
    def fit(self, X, y, sample_weight = None):
        X_Poly = self.poly.fit_transform(X)
        self.linReg.fit(X_Poly, y, sample_weight)
    
    def predict(self, X):
        X_Poly = self.poly.fit_transform(X)
        return self.linReg.predict(X_Poly)

In [31]:
learners = [
 
    SLearner(LinearRegression()),
 
    TLearner(LinearRegression()),
    
    SLearner(PolyLearner(2)),
    
    TLearner(PolyLearner(2))

    #DragonNet(),
 
    #XLearner(LinearRegression()),
 
    #SLearner(RandomForestRegressor()),
 
    #TLearner(RandomForestRegressor()),
 
    #XLearner(RandomForestRegressor()),
 
    #CausalForest()
 
]

In [32]:
from justcause.evaluation import evaluate_ite
 
from justcause.metrics import pehe_score, mean_absolute, bias
 
results = evaluate_ite(expo, learners, metrics=[pehe_score, mean_absolute, bias], random_state=0)

In [33]:
results


[{'pehe_score-mean': 0.9449291908371653,
  'pehe_score-median': 0.9439641308581801,
  'pehe_score-std': 0.003253476504211816,
  'mean_absolute-mean': 0.06754039811889061,
  'mean_absolute-median': 0.06509340724217205,
  'mean_absolute-std': 0.038844906506822936,
  'bias-mean': 0.06551530411720792,
  'bias-median': 0.06507799285709227,
  'bias-std': 0.04217080822099439,
  'method': 'SLearner(learner=LinearRegression)',
  'train': True},
 {'pehe_score-mean': 0.9498430041791069,
  'pehe_score-median': 0.9489510433072321,
  'pehe_score-std': 0.003610202518156286,
  'mean_absolute-mean': 0.07851991278916062,
  'mean_absolute-median': 0.07779388579180013,
  'mean_absolute-std': 0.03991485345896026,
  'bias-mean': -0.07733172159635261,
  'bias-median': -0.07776903285646827,
  'bias-std': 0.04217080822099439,
  'method': 'SLearner(learner=LinearRegression)',
  'train': False},
 {'pehe_score-mean': 0.19710119614063779,
  'pehe_score-median': 0.1955475285090095,
  'pehe_score-std': 0.01537544528

In [34]:
import pandas as pd
df = pd.DataFrame(results).sort_values('pehe_score-mean')
df[df['train'] == False][['method', 'train', 'pehe_score-mean']]

,method,train,pehe_score-mean
3,"TLearner(control=LinearRegression, treated=Lin...",False,1.682366e-01
1,SLearner(learner=LinearRegression),False,9.498430e-01
5,SLearner(learner=PolyLearner),False,4.449764e+09
7,"TLearner(control=PolyLearner, treated=PolyLear...",False,2.724648e+11
